# Prepare data for Training

A number of routines for preparing data inside ML Studio for training

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

#todo: replace file name with the bigger one when script is ready
sales =  pd.DataFrame.from_csv('PE-TRG-Jan-Mar-2017.csv').reset_index()

#remove duplications by summing up quantity by days
sales = pd.pivot_table(sales, values='Quantity', index=['Locationid','PLU','Year','Month','Day','WeekDay'], aggfunc=np.sum).reset_index()

## Raw data properties

In [ ]:
def setCountGroup(row):
    if row['counts'] < 10:
        return "1. less than 10"
    elif row['counts'] <= 20:
        return "2. 20 or less"
    elif row['counts'] <= 30:
        return "3. 30 or less"
    else: 
        return "4. more than 30"

positions = sales.groupby(['Locationid','PLU']).size().rename('counts').reset_index()
positions_count = len(positions.index)
locations_count = len(positions.drop_duplicates(['Locationid']).index)
products_count = len(positions.drop_duplicates(['PLU']).index)

counts_groups = positions
counts_groups['segment'] = counts_groups.apply(setCountGroup, axis=1)
counts_groups = counts_groups.groupby('segment').size().rename('counts').reset_index()
counts_groups['%'] = counts_groups.apply(lambda r: r['counts']/positions_count*100, axis=1)

rows_total = len(sales.index)
rows_with_negative = len(sales[sales['Quantity'] < 0].index)
rows_with_zero = len(sales[sales['Quantity'] == 0].index)
rows_with_positive = len(sales[sales['Quantity'] > 0].index)

# ==== output results ====

print("Total unique positions (Location-Recipe) %d" % (positions_count))
print("Total locations %d" % (locations_count))
print("Total recipes %d" % (products_count))

print("Data rows %d" % (rows_total))
print("Negative data %.2f%% (%d)" % (rows_with_negative/rows_total*100, rows_with_negative))
print("Zero data %.2f%% (%d)" % (rows_with_zero/rows_total*100, rows_with_zero))
print("Positive data %.2f%% (%d)" % (rows_with_positive/rows_total*100, rows_with_positive))

counts_groups

## Prepare data

Set zero values for negative quantity

In [ ]:
#fast version of assignment column value (instead of df.apply)
sales.loc[sales['Quantity'] < 0, 'Quantity'] = 0

Add Saledate column

In [ ]:
date_min = dt.date(sales['Year'].min(),sales['Month'].min(),sales['Day'].min())
date_max = dt.date(sales['Year'].max(),sales['Month'].max(),sales['Day'].max())
dates = pd.DataFrame(pd.date_range(date_min, date_max), columns=['Saledate'])
dates['Year'] = dates['Saledate'].dt.year
dates['Month'] = dates['Saledate'].dt.month
dates['Day'] = dates['Saledate'].dt.day

In [ ]:
sales = pd.merge(sales, dates, on=['Year','Month','Day'], suffixes=('', ''), how='inner')

In [ ]:
#sales = sales.reset_index()
#del sales['index']

### Add Feature: previous 7 days

In [ ]:
pd.options.mode.chained_assignment = None  # disable SettingWithCopyWarning
# slow version becasue it goes row by row
# keep it for history as an example of the slowest approach
def addPreviousDays(p, df, days):
    locationid = p['Locationid']
    plu = p['PLU']
    date_finish = p['Saledate']
    date_start = date_finish - dt.timedelta(days=days)
    # filter sales by position and time frame in days
    position_sales = _sales.loc[(_sales['Locationid']==locationid)
                                &(_sales['PLU']==plu)
                                &(_sales['Saledate']>=date_start)
                                &(_sales['Saledate']<=date_finish)] # at least current row is required to provide the min date
    min_date = position_sales['Saledate'].min()
    
    #add row with starting date if missing
    if min_date > date_start:
        new_row = pd.DataFrame([[locationid,plu,0,0,0,0,date_start]], columns=['Locationid','PLU','Year','Month','Day','Quantity','Saledate'])
        position_sales = position_sales.append(new_row)
    
    position_sales = position_sales.reset_index().set_index('Saledate')
    # Convert the integer timestamps in the index to a DatetimeIndex
    position_sales.index = pd.to_datetime(position_sales.index)
    
    #resample data by days between starting and ending dates
    qty_by_days = position_sales.resample('d').sum().reset_index()
    del qty_by_days['index']
    qty_by_days['Locationid'] = locationid
    qty_by_days['PLU'] = plu
    qty_by_days = qty_by_days.fillna(0)
    
    #transpose resampled data to get dates in columns
    for index in range(1, days+1): # the index is moved +1 to skip the first date in resampled data
        d = date_finish - dt.timedelta(days=index)
        # take the value for day in the past
        # df[].item() - this returns the first element in the Index/Series returned from that selection
        dn = qty_by_days.loc[qty_by_days['Saledate']==d, 'Quantity'].item()
        df.loc[(df['Locationid']==locationid)&(df['PLU']==plu)&(df['Saledate']==date_finish),'Day-'+str(index)] = dn

In [ ]:
# return history of sales using resampling by days
# slowest version
def getSalesHistoryByDays(df, startDate):
    #group by positions and set the starting date with zero quantity
    #the date should be as earliest as possible to not overwrite quantity
    startingPoint = df.groupby(['Locationid','PLU']).sum().reset_index()
    startingPoint['Quantity'] = 0
    startingPoint['Saledate'] = startDate

    #merge position starting points with existing sales
    sale_history = pd.concat([df, startingPoint])
    sale_history = sale_history.reset_index().set_index('Saledate')
    
    #resemble history by groups starting from the start date
    sale_history.index = pd.to_datetime(sale_history.index)
    # (!) the slowest part of the execution
    sale_history = sale_history.groupby(['Locationid','PLU']).resample('d').sum()
    
    #remove duplicated columns
    del sale_history['PLU']
    del sale_history['Locationid']
    sale_history = sale_history.fillna(0)
    
    return sale_history

In [ ]:
# return history of sales using resampling by days
def getSalesHistoryByDays2(df, startDate):
    
    sale_history = df
    
    #init buffer for missing dates
    missing_dates = df.groupby(['Locationid','PLU']).size().reset_index()[['Locationid','PLU']] #size() is used becasue it's fast
    missing_dates['Saledate'] = dt.datetime.now()
    missing_dates = missing_dates[missing_dates['Locationid'] == -1] # selecting nothing to remove rows
    
    # create a buffer with all required dates for all positions
    # simply going by dates and accumulate in one dataframe with all required dates
    for index, row in dates.iterrows():
        date = row['Saledate']
        # create new dataframe to set the next date
        positions = df.groupby(['Locationid','PLU']).size().reset_index()
        positions['Quantity'] = 0
        positions['Saledate'] = date

        missing_dates = pd.concat([missing_dates, positions])
    
    # concat existing sales with rows with missing dates
    sale_history = pd.concat([sale_history, missing_dates])
    sale_history = sale_history.reset_index().set_index('Saledate')
    
    del sale_history['index']
    sale_history = sale_history.fillna(0)
    
    sale_history = sale_history.groupby(['Locationid','PLU','Saledate']).sum().reset_index()
    
    return sale_history

In [ ]:
date_min = sales['Saledate'].min()
# 1 loop, best of 3: 1min 51s per loop
history = getSalesHistoryByDays2(sales, date_min)

In [ ]:
# add one day to the row from the past
# quick version
def addPreviousDay(sales, salesHistory, offsetToPastInDays):
    #definenew key in history frame for later merge
    shiftedHistory = salesHistory
    shiftedHistory = shiftedHistory.reset_index()
    shiftedHistory['D'] = shiftedHistory['Saledate']
    
    #define shifted date-key for merging past history to the same date entry
    df = sales
    df['D'] = df['Saledate'] - dt.timedelta(days=offsetToPastInDays)
    
    #set indexes
    shiftedHistory = shiftedHistory.reset_index().set_index(['Locationid','PLU','D'])
    df = df.reset_index().set_index(['Locationid','PLU','D'])
    
    # merge with history
    df = pd.merge(shiftedHistory, df, left_index=True, right_index=True, suffixes=('_d', ''), how='inner')
    df = df.reset_index()
    
    # cleanup the frame
    df = df.drop(['D','level_0','Saledate_d','index_d','Year_d','Month_d','Day_d','index'], axis=1)
    df = df.rename(columns={'Quantity_d':'Day-'+str(offsetToPastInDays)})
    
    return df

In [ ]:
# add several days from the past
def addPreviousDays(sales, salesHistory, offsetToPastInDays):
    s = sales
    for index in range(1, offsetToPastInDays+1):
        s = addPreviousDay(s, salesHistory, index)
    return s

In [ ]:
sales_with_history = addPreviousDays(sales, history, 7)
sales_with_history

In [ ]:
%timeit addPreviousDays(sales, history, 7)

#take the previous value to add more information (shift - move pointer backward)
qty_by_days['Day-1'] = qty_by_days['Quantity'].shift(1)
qty_by_days['Day-2'] = qty_by_days['Quantity'].shift(2)
qty_by_days['Day-3'] = qty_by_days['Quantity'].shift(3)
qty_by_days['Day-4'] = qty_by_days['Quantity'].shift(4)
qty_by_days['Day-5'] = qty_by_days['Quantity'].shift(5)
qty_by_days['Day-6'] = qty_by_days['Quantity'].shift(6)
qty_by_days['Day-7'] = qty_by_days['Quantity'].shift(7)
#replace NaN with zeros
qty_by_days.fillna(0,inplace=True)

#take weeks averages
def week_avg(df, newColumn, startIndex):
    df[newColumn] = (np.nan_to_num(df['Quantity'].shift(startIndex))
                         +np.nan_to_num(df['Quantity'].shift(startIndex+1))
                         +np.nan_to_num(df['Quantity'].shift(startIndex+2))
                         +np.nan_to_num(df['Quantity'].shift(startIndex+3))
                         +np.nan_to_num(df['Quantity'].shift(startIndex+4))
                         +np.nan_to_num(df['Quantity'].shift(startIndex+5))
                         +np.nan_to_num(df['Quantity'].shift(startIndex+6)))/7
    
week_avg(qty_by_days, 'Week-1', 1)
week_avg(qty_by_days, 'Week-2', 2)
week_avg(qty_by_days, 'Week-3', 3)
week_avg(qty_by_days, 'Week-4', 4)
week_avg(qty_by_days, 'Week-5', 5)

qty_by_days

qty_by_days.corr()['Quantity']

qty_by_days[qty_by_days['PLU'] == 1894].to_csv('1894.csv', sep=',')

import pylab as plt

X = qty_by_days['Quantity']
Y1 = qty_by_days['Day-1']
Y2 = qty_by_days['WeekDay']

#plt.scatter(X,Y1,color='k', s=2)
plt.scatter(X,Y2,color='g', s=2)
plt.show()